In [1]:
def stack(a, b):
    pre = dict(hand = a, isTop = dict(a = False, b = True), on = dict(a = "hand"))
    effect = dict(hand = "", isTop = dict(a = True, b = False), on = dict(a = b))
    name = "stack" + a + b
    return makeAction(name, pre, effect)

def pickup(a):
    pre = dict(hand = "", isTop = dict(a = True), on = dict(a = "table"))
    effect = dict(hand = a, isTop = dict(a = False), on = dict(a = "hand"))
    name = "pickup" + a
    return makeAction(name,pre, effect)

def putdown(a):
    pre = dict(hand = a, isTop = dict(a = False), on = dict(a = "hand"))
    effect = dict(hand = "", isTop = dict(a = True), on = dict(a = "table"))
    name = "putdown" + a
    return makeAction(name, pre, effect)

def unstack(a, b):
    pre = dict(hand = a, isTop = dict(a = False, b = True), on = dict(a = b))
    effect = dict(hand = a, isTop= dict(a = False, b = True), on = dict(a = "hand"))
    name = "unstack" + a + b
    return makeAction(name, pre, effect)
    
def makeAction(name, precondition, effect, cost = 1):
    return dict(name = name, precondition = precondition, effect = effect, cost = cost)

In [2]:
class Node:
    def __init__(self, state, cost, act=0, nxt=0, score=0, diff=None, diff_count=0):
        self.state = state
        self.act = act
        self.nxt = nxt
        self.cost = cost
        self.score = score
        self.diff = diff
        self.diff_count = diff_count
    

In [3]:
domain = [stack("a", "b"),
          stack("a", "c"),
          stack("a", "c"),
          stack("a", "d"),
          stack("b", "c"),
          stack("b", "d"),
          stack("c", "d"),
          unstack("a", "b"),
          unstack("a", "c"),
          unstack("a", "c"),
          unstack("a", "d"),
          unstack("b", "c"),
          unstack("b", "d"),
          unstack("c", "d"),
          pickup("a"),
          pickup("b"),
          pickup("c"),
          pickup("d"),
          putdown("a"),
          putdown("b"),
          putdown("c"),
          putdown("d")]
#print(len(domain))
#print("list の番号　listの中身")
#for i, a in enumerate(domain):
#    print(i,a)
#print("effectのアクセス")
#for l in range(len(domain)):
#    for key, var in domain[l][1].items():
#        print("---key---")
#        print(key)
#        print("---var--")
#        print(var)
#        if key == 'effect':
#            print("-------effect-------")
#            for state_key, state_var in var.items():
#                print("---state_key---")
#                print(state_key)
#                print("---state_var--")
#                print(state_var)
                
            

In [8]:
def getDiffState(dest, src):
    diff = {}
    count = 0
    for key,var in dest.items():
        if type(var) == "dict":
            tbl, cnt = getDiffState(var, src[key])
            if cnt > 0:
                diff[key] = tbl
                count = count + cnt
        elif var != src[key]:
            diff[key] = var
            count = count +1
    return diff, count
def isInState_internal(a, b):
    same_state_count = 0
    for key, var in a.items():
        #print("---key---")
        #print(key)
        #print("---var--")
        #print(var)
        if key == 'effect':
            #print("-------effect-------")
            for state_key, state_var in var.items():
                #print("---state_key---")
                #print(state_key)
                #print("---state_var--")
                #same_state_count = same_state_count +print(state_var)
                if type(state_var) == 'dict':
                    result = isInState_internal(var, b[state_key])
                    if result == False:
                        return False
                    same_state_count = same_state_count + 1
                elif b.get(state_key) != None and state_var != b[state_key]:
                    return False
                elif b.get(state_key) != None and state_var == b[state_key]:
                    same_state_count = same_state_count + 1
    return same_state_count

def isInState(a, b):
    result = isInState_internal(a, b)
    print("----result-----")
    print(result)
    if result != False and result > 0:
        return True
    else:
        return False
    
def getNeighborActions(domain, state):
    actions = []
    for i, d in enumerate(domain):
        if isInState(d, state):
            actions.append(d)
    print(actions)
    return actions

def table_merge(a, b):
    for key, var in b.items():
        if type(var) == 'dict':
            if a[key] == "":
                a[key] = ""
            else:
                a[key] = var
    return a

def getIndex(lst):
    print("*********")
    for l in range(len(lst)):
        print(lst[l])
    print("*********")

In [9]:
def searchPlan(domain, state, goal):
    openlist = []
    closelist = []
    goal_node = Node(state = goal, cost = 0)
    goal_node.diff, goal_node.diffcount =getDiffState(goal_node.state, state)
    goal_node.score = goal_node.diffcount
    openlist.append(goal_node)
    
    while len(openlist) > 0:
        node = openlist.pop(0)
        
        if node.diffcount  == 0:
            return node
        
        closelist.append(node)
        
        neighbors = getNeighborActions(domain, node.state)
        print(neighbors)
        for key, var in neighbors:
            new_state = table_merge(getDiffState(node.state, var.effect), var.precondition)
            diff, diffcount = getDiffState(new_state, state)
            new_cost = node.cost + var.cost
            new_score = new_cost + diffcount
            new_node = Node(state = new_state, act = var, nxt = node, cost = new_cost, socre = new_score, diff = diff, diff_count = diffcount)
            getIndex(openlist)
            
state = dict(hand = "", on = dict(a = "b", b = "table"), isTop = dict(a = True, b = False))
goal = dict(on = dict(a = "table", b = "a"))
searchPlan(domain, state, goal)

----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
----result-----
False
[]
[]
